In [ ]:
import panel as pn
import datetime as dt
import pandas as pd
pn.extension()

# Wired Web Component Example

The [Wired JS](https://wiredjs.com/) widgets provides a set of common UI elements with a hand-drawn, sketchy look. 
These can be used for wireframes, mockups, or just the fun hand-drawn look. 

<img src="https://camo.githubusercontent.com/b399ff5eef4c55afd9ad6c3195b1294e41afe3b9/68747470733a2f2f7062732e7477696d672e636f6d2f6d656469612f44595837782d76577341456863544a2e6a7067" style="width:67%;border:solid">

You might combine the Wired components with 

- sketchy plots generated by [matplotlib.pyplot.xkcd](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.xkcd.html) or
- sketchy drawings via [rough.js](https://roughjs.com/)

You can also find inspiration at the [Wired JS Showcase](https://wiredjs.com/showcase.html).

## Implementation

We implement the Wired components using the `pn.pane.WebComponent`. 

In [ ]:
"""Implementation of the Wired WebComponents"""
import ast
import datetime
import json
from typing import Set, Optional

import param

from panel.pane import WebComponent

JS_FILES = {
    "webcomponents-loader": "https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js",
    "wired-bundle": "https://wiredjs.com/dist/showcase.min.js",
}

ELEVATION_DEFAULT = 0
ELEVATION_BOUNDS = (0, 10)

DATE_BOUNDS = (datetime.date(1976, 9, 17), datetime.date(datetime.datetime.now().year + 10, 12, 31))

### The Wired Javascript library

In [ ]:
%%HTML
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js"></script>
<script src="https://wiredjs.com/dist/showcase.min.js"></script>

#### MWC Icons

The wired components uses the MWC Icons in some of the components and list of `MWC_ICONS` is also defined.

Only a short list is provided here for illustration. You can find the full list [here](https://github.com/google/material-design-icons/blob/master/iconfont/codepoints)

In [ ]:
MWC_ICONS = [
    "3d_rotation",
    "ac_unit",
    "access_alarm",
    "access_alarms",
    "access_time",
    "accessibility",
    "accessible",
    "favorite",
]

### Section Helper Functionality

The `section` is used for showing the wired components. It lays out the `component` together with an optinal message and it's parameters.

In [ ]:
def section(component, message=None, show_html=False):
        parameterset = set(component._child_parameters())
        if show_html:
            parameterset.add("html")
        for parameter in component.parameters_to_watch:
            parameterset.add(parameter)

        parameters = list(parameterset)
        if message:
            return (
                component,
                pn.Param(component, parameters=parameters),
                pn.pane.Markdown(message),
            )
        return pn.Column(component, pn.Param(component, parameters=parameters), pn.layout.Divider())

### WiredBase

The `WiredBase` provides a `Base` class that many of the components will inherit from

In [ ]:
class WiredBase(WebComponent):
    """Inherit from this class"""

    def __init__(self, **params):
        if not self.param.attributes_to_watch.default:
            self.param.attributes_to_watch.default = {}
        self.attributes_to_watch["disabled"] = "disabled"

        super().__init__(**params)

    def _child_parameters(self):
        parameters = super()._child_parameters()
        parameters.add("disabled")
        return parameters

## Button

In [ ]:
class Button(WiredBase):
    """A Wired RadioButton

    - You can set the `text` shown via the `name` parameter.
    """

    html = param.String("<wired-button></wired-button>")
    attributes_to_watch = param.Dict({"elevation": "elevation"})
    events_to_watch = param.Dict(default={"click": "clicks"})
    parameters_to_watch = param.List(["name"])

    clicks = param.Integer()
    elevation = param.Integer(ELEVATION_DEFAULT, bounds=ELEVATION_BOUNDS)

    def __init__(self, **params):
        if "height" not in params:
            params["height"]=40
        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-button>{params['name']}</wired-button>"

### Button Example

In [ ]:
button = Button(name="Allo Panel")
section(button)

### Button Example with `pn.Param`

In [ ]:
class ButtonParameterizedClass(param.Parameterized):
    action = param.Action(label="Click Me")
    clicks = param.Integer(default=0)
    
    def __init__(self, **params):
        super().__init__(**params)
        
        self.action = self._update
    
    def _update(self, event):
        self.clicks += 1
    
button_parameter_class = ButtonParameterizedClass()
pn.Column(
    pn.Param(button_parameter_class, widgets={"action": Button}, show_name=False)
)

## CheckBox

In [ ]:
class Checkbox(WiredBase):
    html = param.String("<wired-checkbox></wired-checkbox>")
    properties_to_watch = param.Dict({"checked": "value"})
    parameters_to_watch = param.List(["name"])

    value = param.Boolean()

    def __init__(self, **params):
        if "height" not in params:
            params["height"]=40

        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-checkbox>{params['name']}</wired-checkbox>"

### CheckBox example

In [ ]:
checkbox=Checkbox(name="Check Me")
section(checkbox)

### CheckBox Example with `pn.Param`

In [ ]:
class CheckBoxParameterizedClass(param.Parameterized):
    panel_is_awesome = param.Boolean(default=True)
    
checkbox_parameter_class = CheckBoxParameterizedClass()
pn.Column(
    pn.Param(checkbox_parameter_class, widgets={"panel_is_awesome": Checkbox}, show_name=False),
    pn.Param(checkbox_parameter_class, show_name=False)
)

## DatePicker

In [ ]:
class DatePicker(WiredBase):
    component_type = param.String("inputgroup")
    html = param.String(
        '<wired-calendar initials="" role="dialog tabindex="0">Button</wired-calendar>'
    )
    attributes_to_watch = param.Dict(
        {
            "elevation": "elevation",
            "firstdate": "firstdate",
            "lastdate": "lastdate",
            "locale": "locale",
        }
    )

    properties_to_watch = param.Dict({"selected": "selected"})
    events_to_watch = param.Dict(default={"selected": "selects"})

    elevation = param.Integer(ELEVATION_DEFAULT, bounds=ELEVATION_BOUNDS)
    # Todo: Support more datatime datahandling instead of strings if possible.
    firstdate = param.String(
        doc="""
    Example: firstdate="Apr 15, 2019"""
    )
    lastdate = param.String(
        doc="""
    Example: lastdate="Jul 15, 2019"""
    )
    locale = param.ObjectSelector("en", objects=["en", "fr", "de"])
    selected = param.String(
        doc="""
    Example: selected="Jul 4, 2019"""
    )
    selects = param.Integer(bounds=(0, None))
    value = param.Date(default=None, bounds=DATE_BOUNDS)
    start = param.Date(bounds=DATE_BOUNDS)
    end = param.Date(bounds=DATE_BOUNDS)

    def __init__(self, min_height=340, min_width=300, **params):
        super().__init__(min_height=min_height, min_width=min_width, **params)

    @staticmethod
    def _to_date(value: Optional[str]) -> Optional[datetime.date]:
        """Converts a date string to a date

        Parameters
        ----------
        value : str
            The str date value
        """
        if value:
            return datetime.datetime.strptime(value, "%b %d, %Y").date()
        return None

    @staticmethod
    def _to_string(value: datetime.date) -> str:
        """Converts a date to a string

        Parameters
        ----------
        value : date
            The date value to convert
        """
        if value:
            return value.strftime("%b %e, %Y").replace("  ", " ")
        return None

    @param.depends("selected", watch=True)
    def _set_value(self):
        value = self._to_date(self.selected)
        if value != self.value:
            self.value = value

    @param.depends("value", watch=True)
    def _set_selected(self):
        selected = self._to_string(self.value)
        if selected != self.selected:
            self.selected = selected

    @param.depends("firstdate", watch=True)
    def _set_start(self):
        start = self._to_date(self.firstdate)
        if start != self.start:
            self.start = start

    @param.depends("start", watch=True)
    def _set_firstdate(self):
        firstdate = self._to_string(self.start)
        if firstdate != self.firstdate:
            self.firstdate = firstdate

    @param.depends("lastdate", watch=True)
    def _set_end(self):
        end = self._to_date(self.lastdate)
        if end != self.end:
            self.end = end

    @param.depends("end", watch=True)
    def _set_lastdate(self):
        lastdate = self._to_string(self.end)
        if lastdate != self.lastdate:
            self.lastdate = lastdate

#### Known Issues

- For some reason you cannot update the `locale` dynamically. You can set it in instantiation though.

### DatePicker Example

In [ ]:
datepicker = DatePicker()
section(datepicker)

### DatePicker Example with `pn.Param`

In [ ]:
class DatePickerParameterizedClass(param.Parameterized):
        date = param.Date(
            dt.date(2017, 1, 1), bounds=DATE_BOUNDS,
        )
date_parameterized_class = DatePickerParameterizedClass()
pn.Column(
    pn.Param(date_parameterized_class, widgets={"date": DatePicker}, show_name=False),
    pn.Param(date_parameterized_class, show_name=False)
)

### Known Issues

- Cannot set date. Get

```bash
Python failed with the following traceback: 
c:\repos\private\panel\.venv\lib\site-packages\pyviz_comms\__init__.py _handle_msg L315
c:\repos\private\panel\.venv\lib\site-packages\panel\viewable.py _on_msg L242
c:\repos\private\panel\.venv\lib\site-packages\bokeh\document\document.py unhold L669
c:\repos\private\panel\.venv\lib\site-packages\bokeh\document\document.py _trigger_on_change L1132
c:\repos\private\panel\.venv\lib\site-packages\bokeh\document\document.py _with_self_as_curdoc L1150
c:\repos\private\panel\.venv\lib\site-packages\bokeh\util\callback_manager.py invoke L155
c:\repos\private\panel\.venv\lib\site-packages\panel\viewable.py _comm_change L494
c:\repos\private\panel\.venv\lib\site-packages\panel\viewable.py _process_events L511
c:\repos\private\panel\.venv\lib\site-packages\panel\widgets\input.py _process_property_change L332
	TypeError: can't compare datetime.datetime to datetime.date
```

## Divider

In [ ]:
class Divider(WebComponent):
    html = param.String("<wired-divider></wired-divider>")

    def __init__(self, min_height=20, **params):
        super().__init__(min_height=min_height, **params)

    attributes_to_watch = param.Dict({"elevation": "elevation"})
    elevation = param.Integer(ELEVATION_DEFAULT, bounds=ELEVATION_BOUNDS)

### Divider Example

In [ ]:
divider=Divider()
section(divider)

## Fab

In [ ]:
class Fab(WiredBase):
    html = param.String("<wired-fab><mwc-icon>favorite</mwc-icon></wired-fab>")
    parameters_to_watch = param.List(["icon"])

    icon = param.ObjectSelector(
        "favorite",
        objects=MWC_ICONS,
        doc="""
    The name of an `mwc-icon <https://github.com/material-components/material-components-web-components/tree/master/packages/icon>`_
    """,
    )

    def __init__(
        self, min_height=40, **params,
    ):
        super().__init__(min_height=min_height, **params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-fab><mwc-icon>{params['icon']}</mwc-icon></wired-fab>"

### Fab Example

In [ ]:
fab = Fab()
section(fab)

## IconButton

In [ ]:
class IconButton(WiredBase):
    html = param.String("<wired-icon-button><mwc-icon>favorite</mwc-icon><wired-icon-button>")
    parameters_to_watch = param.List(["icon"])
    events_to_watch = param.Dict(default={"click": "clicks"})

    icon = param.ObjectSelector(
        "favorite",
        objects=MWC_ICONS,
        doc="""
    The name of an `mwc-icon <https://github.com/material-components/material-components-web-components/tree/master/packages/icon>`_
    """,
    )
    clicks = param.Integer()

    def __init__(
        self, min_height=40, **params,
    ):
        super().__init__(min_height=min_height, **params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-icon-button><mwc-icon>{params['icon']}</mwc-icon></wired-icon-button>"

### IconButton Example

In [ ]:
icon_button = IconButton()
section(icon_button)

### IconButton Example with `pn.Param`

In [ ]:
class IconButtonParameterizedClass(param.Parameterized):
    action = param.Action(label="Click Me")
    clicks = param.Integer(default=0)
    
    def __init__(self, **params):
        super().__init__(**params)
        
        self.action = self._update
    
    def _update(self, event):
        self.clicks += 1
    
icon_button_parameter_class = IconButtonParameterizedClass()
pn.Column(
    pn.Param(icon_button_parameter_class, widgets={"action": IconButton}, show_name=False)
)

## Image

In [ ]:
class Image(WebComponent):
    """The wired-image element"""

    html = param.String('<wired-image style="width:100%;height:100%"></wired-image>')
    attributes_to_watch = param.Dict({"elevation": "elevation", "src": "object", "alt": "alt_text"})

    # @Philippfr: How do I handle height and width in general in the .ts model?
    def __init__(self, height=100, **params):
        super().__init__(height=height, **params)

    object = param.String(default=None, doc="""Currently only an url is supported""")
    alt_text = param.String(default=None)
    elevation = param.Integer(ELEVATION_DEFAULT, bounds=ELEVATION_BOUNDS)

### Image Example

In [ ]:
image = Image(object="https://panel.holoviz.org/_static/logo_stacked.png", width=100)
section(image)

## Link

In [ ]:
class Link(WebComponent):
    html = param.String("<wired-link></wired-link>")
    attributes_to_watch = param.Dict({"href": "href", "target": "target"})
    parameters_to_watch = param.List(["text"])

    href = param.String()
    target = param.ObjectSelector("_blank", objects=["_self", "_blank", "_parent", "_top"])
    text = param.String()

    def __init__(self, **params):
        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-link>{params['text']}</wired-link>"

### Link Example

In [ ]:
link = Link(text="awesome-panel.org", href="https://awesome-panel.org")
section(link)

## Progress

In [ ]:
class Progress(WebComponent):
    html = param.String("<wired-progress></wired-progress>")
    attributes_to_watch = param.Dict({"value": "value", "percentage": "percentage", "max": "max"})

    def __init__(self, min_height=40, **params):
        super().__init__(min_height=min_height, **params)

        if "max" in params:
            self._handle_max_changed()

    value = param.Integer(None, bounds=(0, 100))
    max = param.Integer(100, bounds=(0, None))
    percentage = param.Boolean()

    @param.depends("max", watch=True)
    def _handle_max_changed(self):
        self.param.value.bounds = (0, self.max)

### Progress Example

In [ ]:
progress = Progress(value=33)
section(progress)

## RadioButton

In [ ]:
class RadioButton(WebComponent):
    """A Wired RadioButton"""

    html = param.String("<wired-radio>Radio Button</wired-radio>")
    properties_to_watch = param.Dict({"checked": "value"})
    parameters_to_watch = param.List(["name"])

    value = param.Boolean(default=False)
    
    def __init__(self, **params):
        if "height" not in params:
            params["height"]=15
        super().__init__(**params)

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        return f"<wired-radio>{params['name']}</wired-radio>"

### RadioButton Example

In [ ]:
radiobutton = RadioButton(name="Check Me")
section(radiobutton)

### RadioButton Example with `pn.Param`

In [ ]:
class RadioButtonParameterizedClass(param.Parameterized):
    checked = param.Boolean(default=True)
    
radio_button_parameterized_class = RadioButtonParameterizedClass()
pn.Column(
    pn.Param(radio_button_parameterized_class, widgets={"checked": RadioButton}, show_name=False),
    pn.Param(radio_button_parameterized_class, show_name=False)
)

In [ ]:
class SearchInput(WiredBase):
    html = param.String("<wired-search-input></wired-search-input>")
    attributes_to_watch = param.Dict({"placeholder": "placeholder", "autocomplete": "autocomplete"})
    properties_to_watch = param.Dict({"textInput.value": "value"})
    events_to_watch = param.Dict({"input": None})

    def __init__(self, min_height=40, **params):
        if "value" in params:
            self.param.html.default = f'<wired-search-input value="{params["value"]}" style="width:100%;height:100%"></wired-search-input>'
        elif self.param.value.default:
            self.param.html.default = f'<wired-search-input value="{self.param.value.default}" style="width:100%;height:100%"></wired-search-input>'

        super().__init__(min_height=min_height, **params)

    placeholder = param.String("")
    value = param.String()
    autocomplete = param.ObjectSelector("off", objects=["on", "off"])


# Todo: Implement Tabs. It's really a layout. Don't yet know how to support this.

# @Philippfr: Should we merge the wired Progress and wired ProgressSpinner into one with
# functionality similar to the Panel Progress?
class ProgressSpinner(WebComponent):
    html = param.String("<wired-spinner></wired-spinner>")
    attributes_to_watch = param.Dict({"spinning": "active", "duration": "duration"})

    active = param.Boolean(default=True)
    duration = param.Integer(default=1000, bounds=(1, 10000))

    def __init__(self, min_height=40, **params):
        super().__init__(min_height=min_height, **params)

class TextAreaInput(WiredBase):
    component_type = param.String("inputgroup")
    html = param.String('<wired-textarea placeholder="Enter text"></wired-textarea>')
    attributes_to_watch = param.Dict({"placeholder": "placeholder"})
    properties_to_watch = param.Dict({"textareaInput.value": "value", "rows": "rows", "maxlength": "max_length"})
    events_to_watch = param.ObjectSelector(
        {"change": None},
        objects=[{"change": None}, {"input": None}],
        doc="""
    The event(s) to watch. When the event(s) are catched the js model properties are checked and
    any changed values are sent to the python model. The event can be
    - `change` (when done) or
    - `input` (for every character change)
    """,
    )

    placeholder = param.String("")
    value = param.String()
    rows = param.Integer(2, bounds=(1, 100))
    max_length = param.Integer(default=5000)

    def __init__(self, **params):
        # if "value" in params:
        #     self.param.html.default = f'<wired-textarea value="{params["value"]}"></wired-textarea>'
        # elif self.param.value.default:
        #     self.param.html.default = f'<wired-textarea value="{self.param.value.default}"></wired-textarea>'

        super().__init__(**params)

        if not "min_height" in params:
            self._set_height()

    @param.depends("rows", "disabled", watch=True)
    def _set_height(self):
        height = 20 + 19 * self.rows
        if self.disabled:
            height += 4
        if height != self.height:
            self.height = height


class TextInput(WiredBase):
    component_type = param.String("inputgroup")
    html = param.String("""<wired-input style="width:100%;height:100%"></wired-input>""")
    attributes_to_watch = param.Dict(
        {
            "placeholder": "placeholder",
            "type": "type_",
            # "min": "start",
            # "max": "end",
            # "step": "step",
        }
    )
    properties_to_watch = param.Dict({"textInput.value": "value"})
    events_to_watch = param.Dict({"change": None})

    # @Philippff. I sthis the right place to define height? And what about width?
    def __init__(self, min_height=50, **params):
        # Hack: To solve https://github.com/wiredjs/wired-elements/issues/123
        if "value" in params:
            self.param.html.default = f'<wired-input value="{params["value"]}" style="width:100%;height:100%"></wired-input>'
        elif self.param.value.default:
            self.param.html.default = f'<wired-input value="{self.param.value.default}" style="width:100%;height:100%"></wired-input>'

        super().__init__(min_height=min_height, **params)

    placeholder = param.String(default="")
    type_ = param.ObjectSelector("", objects=["", "password"])
    value = param.String()
    # start = param.Integer(None)
    # end = param.Integer(None)
    # step = param.Parameter(None)

class Toggle(WiredBase):
    html = param.String("<wired-toggle></wired-toggle>")
    properties_to_watch = param.Dict({"checked": "value"})
    events_to_watch = param.Dict({"change": None})

    def __init__(self, min_height=20, **params):
        super().__init__(min_height=min_height, **params)

    value = param.Boolean(False)


class Select(WebComponent):
    # Todo: Implement api for added wired-combo-items to the innerhtml
    # Todo: The selected attribute/ parameter is not updated. Fix this
    component_type = param.String("inputgroup")
    html = param.String("""<wired-combo></wired-combo>""")
    properties_to_watch = param.Dict({"selected": "value"})
    events_to_watch = param.Dict(default={"selected": "selects"})
    parameters_to_watch = param.List(["options"])

    def __init__(self, min_height=40, **params):
        super().__init__(min_height=min_height, **params)

        self._set_class_()

    value = param.Parameter()
    selects = param.Integer()
    options = param.ClassSelector(default=[], class_=(dict, list))

    def _get_html_from_parameters_to_watch(self, **params) -> str:
        options = params["options"]
        if not options:
            return """<wired-combo></wired-combo>"""

        innerhtml = []
        if isinstance(options, list):
            for obj in options:
                item = f'<wired-item value"{str(obj)}" role="option">{str(obj)}</wired-item>'
                innerhtml.append(item)
        if isinstance(options, dict):
            for key, value in options.items():
                item = f'<wired-item value"{str(key)}" role="option">{str(value)}</wired-item>'
                innerhtml.append(item)

        return f"""<wired-combo>{"".join(innerhtml)}</wired-combo>"""

    # @Phillipfr: Don't understand why this is nescessary. But get error if I don't do it.
    @param.depends("options", watch=True)
    def _set_class_(self):
        if isinstance(self.options, list):
            self.param.options.class_ = list
        if isinstance(self.options, dict):
            self.param.options.class_ = dict


class Video(WebComponent):
    html = param.String(
        """<wired-video autoplay="" playsinline="" muted="" loop="" style="height: 80%;" src="https://file-examples.com/wp-content/uploads/2017/04/file_example_MP4_480_1_5MG.mp4"></wired-video>"""
    )
    attributes_to_watch = param.Dict(
        {"autoplay": "autoplay", "playsinline": "playsinline", "loop": "loop", "src": "object"}
    )

    def __init__(self, min_height=250, margin=50, **params):
        super().__init__(min_height=min_height, **params)

    object = param.String(doc="""Currently only an url is supported""")
    autoplay = param.Boolean()
    playsinline = param.Boolean()
    muted = param.Boolean()
    loop = param.Boolean()


## Not Implemented

- Dialog
- FloatSlider
- IntSlider
- LiteralInput

- RadioBoxGroup
- PasswordInput
